In [1]:
# Standard Libraries
import os
import json
import datetime
import numpy as np
import tensorflow as tf

import time
import nvtx

import argparse

import math
from tensorflow.keras.utils import Progbar

# ------------------------------- CUSTOM FUNCTIONS ------------------------------------------------
# Custom Library
import sys
sys.path.append('../')
    
from proxy_apps.apps.timeseries_prediction import deepDMD, proxyDeepDMD, proxyDeepDMDMGPU, hyperparameters

from proxy_apps.utils.tf import TimingCallback
from proxy_apps.utils.data.main import NpEncoder
from proxy_apps.utils import file_reader, path_handler
from proxy_apps.utils.data.grid import GridNetworkDataHandler, GridNetworkTFDataHandler, GridNetworkNewGen, TransientDataset

In [2]:
import logging

In [3]:
logger = tf.get_logger()

In [4]:
# System Setup
config = file_reader.read_config()

_N_EPOCHS = 2
_BATCH_SIZE = 65536
_APP_NAME = config["info"]["app_name"]
_NROWS = int(config["data"]["n_rows"])
_NCOLS = int(config["data"]["n_cols"])
_REPEAT_COLS = 1 # int(config["data"]["repeat_cols"])
_WINDOW_SIZE = int(config["data"]["window_size"])
_SHIFT_SIZE = int(config["data"]["shift_size"])
_STRIDE = int(config["data"]["stride"])
_N_SIGNALS = int(config["data"]["n_signals"])

_DTYPE = config["model"]["dtype"]

_LABEL = "TFDataOpt"
_SUFFIX =  "gpu" + '_' + \
            "a100" + '_' + \
            'ng' + str(1) + '_' + \
            'nc' + str(-1) + '_' + \
            'e' + str(_N_EPOCHS) + '_' + \
            'b' + str(_BATCH_SIZE) + '_' + \
            'r' + str(_REPEAT_COLS) + '_' + _LABEL

performance_dict = dict()

tic = time.time()

# current directory
curr_dir = "./"

# output directory
output_dir = path_handler.get_absolute_path(curr_dir, config["info"]["output_dir"] + config["info"]["name"] + "/" + config["info"]["app_name"] + "/" + _DTYPE + "/R" + str(_REPEAT_COLS) + "/")
if not os.path.exists(output_dir): os.makedirs(output_dir)

# TensorFlow Setup
logger.info(" [Training Script]: Tensorflow version: %s", tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    log.info("Name:", gpu.name, "  Type:", gpu.device_type)

INFO:tensorflow: [Training Script]: Tensorflow version: 2.4.0


In [5]:
# tf.compat.v1.disable_eager_execution()
logger.info("Eager mode: %s", tf.executing_eagerly()) # For easy reset of notebook state.

tf.keras.backend.clear_session()
tf.keras.backend.set_floatx(_DTYPE)

INFO:tensorflow:Eager mode: True


In [6]:
hyper_param_dict = config["model"]["hyperparameters"]
hyper_param_dict['original_dim']       = _REPEAT_COLS * _NCOLS   # input data dimension
hyper_param_dict['num_epochs']         = _N_EPOCHS  # Number of epochs  
hyper_param_dict['batch_size']         = _BATCH_SIZE

hyper_param_dict['dtype']         = _DTYPE
hp = hyperparameters.HyperParameters(hyper_param_dict)
hp.model_name         = _LABEL

performance_dict["n_epochs"] = hp.ep
performance_dict["batch_size"] = hp.bs

In [7]:
def get_indexer(n_rows, window_size, shift_size, start_point, leave_last):
    return np.arange(window_size)[None, :] + start_point + shift_size*np.arange(((n_rows - window_size - leave_last - start_point) // shift_size) + 1)[:, None]

In [8]:
# ------------------------------- DATA LOADING ------------------------------------------------   
l_start = time.time()
if _LABEL == "Baseline":
    data_handler = GridNetworkDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

    scenario_data = data_handler.load_grid_data()
elif _LABEL == "TFDataOpt":
    data_handler = GridNetworkNewGen(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            d_type=_DTYPE
                                         )

    x_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 0, _N_SIGNALS)
    y_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 1, 0)

    scenario_data = data_handler.get_training_data(x_indexer, y_indexer)
elif _LABEL == "TFDataOptPrev":
    data_handler = GridNetworkTFDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

    scenario_data = data_handler.load_grid_data()

l_stop = time.time()
logger.info(' [Training Script]: Time taken for loading datasets: %f seconds', l_stop - l_start)

INFO:tensorflow: [Training Script]: Time taken for loading datasets: 0.269189 seconds


In [9]:
data_handler = GridNetworkNewGen(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                    n_rows=_NROWS,
                                    n_cols=_NCOLS,
                                    repeat_cols=_REPEAT_COLS,
                                    d_type=_DTYPE
                                 )

yp_indexer = get_indexer(_NROWS, _NROWS-1, _SHIFT_SIZE, 0, 1)
yf_indexer = get_indexer(_NROWS, _NROWS-1, _SHIFT_SIZE, 1, 0)

scenario_data = data_handler.get_training_data(yp_indexer, yf_indexer)

In [10]:
count = 0
for a, b in scenario_data:
    count += 1
print(count)

41970


In [ ]:
get_indexer(_NROWS, 1399, _SHIFT_SIZE, 1, 0)

In [ ]:
if _LABEL in ["Baseline", "TFDataOptPrev"]:
    # ------------------------------- DATA PREPROCESSING ------------------------------------------------
    i_start = time.time()
    X_data, Y_data, U_data, V_data, Yp, Yf = data_handler.create_windows(scenario_data)
    i_stop = time.time()
    logger.info(' [Training Script]: Time taken for creating datasets: %f seconds', i_stop - i_start)
    
    # ------------------------------- DATA NORMALIZATION ------------------------------------------------
    n_start = time.time()
    X_array, Y_array, U_array, V_array, Yp_array, Yf_array = data_handler.scale_data(X_data, Y_data, U_data, V_data, Yp, Yf)
    n_stop = time.time()
    logger.info(' [Training Script]: Time taken for normalization: %f seconds', n_stop - n_start)

In [ ]:
# ------------------------------- MODEL TRAINING ------------------------------------------------
model = proxyDeepDMDMGPU.Encoder(hp)
optimizer = tf.optimizers.Adagrad(hp.lr)

# compile and fit model
model.compile(optimizer=optimizer)

trainer = proxyDeepDMDMGPU.NeuralNetworkModel(hp, model)

In [ ]:
# timing callback
timing_cb = TimingCallback()

In [ ]:
model.build(input_shape=(None, _REPEAT_COLS * _NCOLS))
model.summary()

In [ ]:
if _LABEL == "Baseline":
    m_start = time.time()
    trainer.fit(zip(X_array, Y_array), n_epochs=2, batch_size=_BATCH_SIZE, steps_per_epoch=22)
    m_stop = time.time()
elif _LABEL == "TFDataOpt":
    data_options = tf.data.Options()
    data_options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
    training_dataset = scenario_data.with_options(data_options).batch(hp.bs)

    training_dataset = training_dataset.cache()
    shuffle_buffer_size = x_indexer.shape[0]*x_indexer.shape[1]*len(data_handler.dir_list) // _BATCH_SIZE
    training_dataset = training_dataset.shuffle(buffer_size=x_indexer.shape[0]*x_indexer.shape[1]*len(data_handler.dir_list)//_BATCH_SIZE)
    training_dataset = training_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    m_start = time.time()
    trainer.fit(training_dataset, n_epochs=_N_EPOCHS, batch_size=_BATCH_SIZE, steps_per_epoch=22)
    m_stop = time.time()
elif _LABEL == "TFDataOptPrev":
    # tensorflow dataset conversion
    tf_conv_start = time.time()
    training_dataset = tf.data.Dataset.zip((X_array, Y_array)).batch(_BATCH_SIZE)
    training_dataset = training_dataset.cache()
    training_dataset = training_dataset.shuffle(buffer_size=22)
    training_dataset = training_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    tf_conv_stop = time.time()
    logger.info('[INFO]: Tensorflow Conversion Time: %f seconds', tf_conv_stop - tf_conv_start)
    
    m_start = time.time()
    trainer.fit(training_dataset, n_epochs=_N_EPOCHS, batch_size=_BATCH_SIZE, steps_per_epoch=22)
    m_stop = time.time()
    
# print info
logger.info('[INFO]: Time taken for model training (time module): %f seconds', m_stop - m_start)
logger.info('[INFO]: Time taken for model training (Keras): %f seconds', sum(timing_cb.logs))

In [ ]:
trainer.encoder.losses